## Import main packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

## Import Specific Modules

In [ ]:
# preprocessing
from sklearn.preprocessing import normalize

# classifiers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from logitboost import LogitBoost

from tqdm import tqdm
# model selection
from sklearn.model_selection import StratifiedShuffleSplit

# model evaluation
from sklearn.metrics import confusion_matrix, auc

## Define classifiers, and corresponding parameters

In [ ]:
classifierNames = pd.DataFrame(["kNN", "DT", "AdaBoost","GNB","LR", "LDA","SVM.SVC","LogitBoost"])

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    AdaBoostClassifier(),
    GaussianNB(),
    LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial'),
    # LinearDiscriminantAnalysis(solver='lsqr', shrinkage=0.81),  # rv.kris
    LinearDiscriminantAnalysis(),  # rv.kris
    make_pipeline(StandardScaler(), SVC(gamma='auto')),
    LogitBoost(n_estimators=200, random_state=0)
]

## Define settings for printing, plotting, etc.

In [ ]:
pd.set_option("display.float_format", lambda x: "%.3f" % x)
%matplotlib inline

## Read input data from file
Data contains 3 columns: 'date', 'rain1'(1-day cumulative rainfall), and 'ls' (boolean to indicate if date is a landslide or non-landslide event). Rainfall data, from 2014 to 2018 is from PAGASA Baguio synoptic station, whereas landslide or non-landslide event is determined from the maintenance records of the DPWH.

In [ ]:
# read from csv file, get 'date', 'rain', and 'landslide' columns
rainls = pd.read_csv("DATAhumidity.csv", usecols=list(range(0, 12)))

# format date column
rainls["date"] = pd.to_datetime(rainls["date"])

# show first five rows of dataframe
print(rainls.head(10))

# show dimensions of the dataframe (rows, columns)
rainls.shape

## Create features

With the hypothesis that the landslide occurrences are determined by certain rainfall characteristics, we create features (columns) from the rainfall time series in our input data.  We explore combinations of cumulative and offset functions to generate the various rainfall characteristics.

In [ ]:
# FINAL FEATURE GENERATION
shift = list(range(0, 175))     #(25 weeks) #(Carvajal, T.M., Viacrusis, K.M., Hernandez, L.F.T. et al. Machine learning methods reveal the temporal pattern of dengue incidence using meteorological factors in metropolitan Manila, Philippines. BMC Infect Dis 18, 183 (2018). https://doi.org/10.1186/s12879-018-3066-0)
shiftrain = pd.DataFrame(index=rainls.index)
for i in shift:
    shiftrain[f"shift_{str(i)}"] = rainls["rain1"].shift(-i)
    shiftrain[f"shiftmmaxh_{str(i)}"] = rainls["meanmaxh"].shift(-i)
    shiftrain[f"shiftmminh_{str(i)}"] = rainls["meanminh"].shift(-i)
    shiftrain[f"shiftmeanh_{str(i)}"] = rainls["meanh"].shift(-i)
    shiftrain[f"shiftrh_{str(i)}"] = rainls["relhumh"].shift(-i)
# adding lables to samples (rows)
shiftrain["den"] = rainls.den
# removing NaNs as result of cumulative and lag functions
shiftrain = shiftrain.dropna(axis=0, how="any")
COrain = shiftrain
COrain

## Splitting, training, testing, evaluating

# MEAN MAX MINIMUM TEMPERATURE RELATIVE HUMIDITY

In [ ]:
my_list = list(range(1))        # rv.kris - this increases the number of iterations, set this to 100, but this also multiples the training time by the same amount

# define split function
n_splits = 30
test_size = 0.25
random_state = None             # rv.kris - place your seed here, author used "0" (zero)
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)

# create dataframes for evaluation results
TPRs = pd.DataFrame(index=range(n_splits), columns=classifierNames[0])
FPRs = pd.DataFrame(index=range(n_splits), columns=classifierNames[0])
TNRs = pd.DataFrame(index=range(n_splits), columns=classifierNames[0])
aROCs = pd.DataFrame(index=range(n_splits), columns=classifierNames[0])

# loop through all splits
with tqdm(total=len(my_list)) as pbar:
    for x in my_list:
        for s, (train_index, test_index) in zip(TPRs.index, sss.split(COrain.iloc[:, :-1], COrain.iloc[:, -1])):
            # define training and testing sets using results of the splitting function
            ytrain, ytest = COrain["den"].iloc[train_index], COrain["den"].iloc[test_index]
            Xtrain, Xtest = COrain[COrain.columns[:-1]].iloc[train_index], COrain[COrain.columns[:-1]].iloc[test_index]
            # normalize training set features (column-wise)
            Xtrain_norm, Xnorm = normalize(Xtrain, axis=0, return_norm=True)
            # normalize testing set features (column-wise)
            Xtest_norm = Xtest/Xnorm
            # loop through algorithms
            for c in range(len(classifierNames)):
                # if c in [0, 1, 2, 3, 4, 6, 7, 8]:continue      # skip some algorithms # rv.kris-  the defaults used by the author was 1, 8... we can set this to all except 5 (LDA)...
                if c in [1, 8]:continue      # skip some algorithms
                # print(x, s, c, classifiers[c])
                # define current classifier
                clf = classifiers[c]
                # train the classifier using the training set
                clf.fit(Xtrain_norm, ytrain)
                # predict values using the trained classifier and the test set
                Xtest_norm = np.array(Xtest_norm)         # rv.kris
                y_pred = clf.predict(Xtest_norm)
                # Evaluate the test
                # Define true values
                y_true = ytest
                # Produce confusion matrix from true and predicted values
                tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
                # Compute true positive and false positive rates and assign to dataframe
                tpr = np.round((tp / (tp+fn*1.)), 4)
                fpr = np.round((fp / (fp+tn*1.)), 4)
                tnr = np.round((tn / (tn+fp*1.)), 4)
                err = np.round((fp+fn / (tn+fp+tp+fn*1.)), 4)
                aROC = auc([0, fpr, 1], [0, tpr, 1])
                # Assign results to dataframes
                TPRs.loc[s, classifierNames.values[c][0]] = tpr
                TNRs.loc[s, classifierNames.values[c][0]] = tnr
                FPRs.loc[s, classifierNames.values[c][0]] = fpr
                # ERRs.loc[s, classifierNames.values[c][0]] = err
                aROCs.loc[s, classifierNames.values[c][0]] = aROC
            pbar.update(1)

# Remove NA columns (skipped algorithms)
FPRs = FPRs.dropna(axis=1, how="all")
TPRs = TPRs.dropna(axis=1, how="all")
TNRs = TNRs.dropna(axis=1, how="all")
# ERRs = ERRs.dropna(axis=1, how="all")
aROCs = aROCs.dropna(axis=1, how="all")

## Results

In [ ]:
# my_list = list(range(100))
# convert results to percent (0-100)
_FPRs = FPRs * 100
_TPRs = TPRs * 100
_TNRs = TNRs * 100
# ERRs = ERRs * 100
_aROCs = aROCs * 100

print(_aROCs)

# _aROCs["kNN"] = _aROCs["kNN"].astype(float)                       # rv.kris
# _aROCs["AdaBoost"] = _aROCs["AdaBoost"].astype(float)             # rv.kris
# _aROCs["GNB"] = _aROCs["GNB"].astype(float)                       # rv.kris
# _aROCs["LR"] = _aROCs["LR"].astype(float)                         # rv.kris
_aROCs["LDA"] = _aROCs["LDA"].astype(float)                       # rv.kris
# _aROCs["SVM.SVC"] = _aROCs["SVM.SVC"].astype(float)               # rv.kris
# _aROCs["LogitBoost"] = _aROCs["LogitBoost"].astype(float)         # rv.kris
# _TPRs["kNN"] = _TPRs["kNN"].astype(float)                         # rv.kris
# _TPRs["AdaBoost"] = _TPRs["AdaBoost"].astype(float)               # rv.kris
# _TPRs["GNB"] = _TPRs["GNB"].astype(float)                         # rv.kris
# _TPRs["LR"] = _TPRs["LR"].astype(float)                           # rv.kris
_TPRs["LDA"] = _TPRs["LDA"].astype(float)                         # rv.kris
# _TPRs["SVM.SVC"] = _TPRs["SVM.SVC"].astype(float)                 # rv.kris
# _TPRs["LogitBoost"] = _TPRs["LogitBoost"].astype(float)           # rv.kris
# _FPRs["kNN"] = _FPRs["kNN"].astype(float)                         # rv.kris
# _FPRs["AdaBoost"] = _FPRs["AdaBoost"].astype(float)               # rv.kris
# _FPRs["GNB"] = _FPRs["GNB"].astype(float)                         # rv.kris
# _FPRs["LR"] = _FPRs["LR"].astype(float)                           # rv.kris
_FPRs["LDA"] = _FPRs["LDA"].astype(float)                         # rv.kris
# _FPRs["SVM.SVC"] = _FPRs["SVM.SVC"].astype(float)                 # rv.kris
# _FPRs["LogitBoost"] = _FPRs["LogitBoost"].astype(float)           # rv.kris
# _TNRs["kNN"] = _TNRs["kNN"].astype(float)                         # rv.kris
# _TNRs["AdaBoost"] = _TNRs["AdaBoost"].astype(float)               # rv.kris
# _TNRs["GNB"] = _TNRs["GNB"].astype(float)                         # rv.kris
# _TNRs["LR"] = _TNRs["LR"].astype(float)                           # rv.kris
_TNRs["LDA"] = _TNRs["LDA"].astype(float)                         # rv.kris
# _TNRs["SVM.SVC"] = _TNRs["SVM.SVC"].astype(float)                 # rv.kris
# _TNRs["LogitBoost"] = _TNRs["LogitBoost"].astype(float)           # rv.kris

# HISTOGRAMS
# Define parameters for histograms
bins = 10
stacked = True
figsize = (6, 4)
# Plot histograms
ax1 = _aROCs.plot.hist(stacked=stacked, bins=bins, figsize=figsize)
ax1.legend(ncol=2)
ax1.set_xlabel("aROC, %", fontsize="large")
plt.savefig("aROCshistmeanmaxminrel.png")

ax2 = _TPRs.plot.hist(stacked=stacked, bins=bins, figsize=figsize)
ax2.legend(ncol=2)
ax2.set_xlabel("TPR, %", fontsize="large")
plt.savefig("TPRshistmeanmaxminrel.png")

ax3 = _FPRs.plot.hist(stacked=stacked, bins=bins, figsize=figsize)
ax3.legend(ncol=2)
ax3.set_xlabel("FPR, %", fontsize="large")
plt.savefig("FPRshistmeanmaxminrel.png")

ax7 = _TNRs.plot.hist(stacked=stacked, bins=bins, figsize=figsize)
ax7.legend(ncol=2)
ax7.set_xlabel("TNR, %", fontsize="large")
plt.savefig("TNRshistmeanmaxminrel.png")

# SCATTER PLOT
# Define figure parameters
fig = plt.figure(figsize=(6, 6))
ax4 = fig.add_subplot(111)
marker = "o"
markersize = 10
linewidth = 0
alpha = 0.5
mfc = "None"
mew = 2
# Loop through results per algorithm
with tqdm(total=len(my_list)) as pbar:
    for cN in _FPRs.columns:
        # plot results of current algorithm
        ax4.plot(_FPRs[cN].values, _TPRs[cN].values, marker=marker, mfc=mfc, mew=mew, markersize=markersize, linewidth=linewidth, alpha=alpha, label=cN)
        # Add random guess line
    pbar.update(1)

ax4.plot([0, 100], [0, 100], "k:", label="random guess")
# Format plot
ax4.legend()
ax4.set_ylabel("TPR, %", fontsize="large")
ax4.set_xlabel("FPR, %", fontsize="large")
ax4.set_xlim(-5, 105)
ax4.set_ylim(-5, 105)
plt.savefig("ROCplotMLmeanmaxminrel.png")

In [ ]:
# TABULAR RESULTS
aROCsum = pd.DataFrame()
aROCsum["mean"] = _aROCs.mean()
aROCsum["std"] = _aROCs.std()
aROCsum["min"] = _aROCs.min()
aROCsum["maxn"] = _aROCs.max()

TPRsum = pd.DataFrame()
TPRsum["mean"] = _TPRs.mean()
TPRsum["std"] = _TPRs.std()
TPRsum["min"] = _TPRs.min()
TPRsum["maxn"] = _TPRs.max()

FPRsum = pd.DataFrame()
FPRsum["mean"] = _FPRs.mean()
FPRsum["std"] = _FPRs.std()
FPRsum["min"] = _FPRs.min()
FPRsum["maxn"] = _FPRs.max()

TNRsum = pd.DataFrame()
TNRsum["mean"] = _TNRs.mean()
TNRsum["std"] = _TNRs.std()
TNRsum["min"] = _TNRs.min()
TNRsum["maxn"] = _TNRs.max()

display(TNRsum)
display(FPRsum)
display(TPRsum)
display(aROCsum)

# revert result to percent (0.0 - 1.0)
# FPRs = FPRs/100
# TPRs = TPRs/100
# aROCs = aROCs/100
# TNRs = TNRs/100

## Comparison with traditional cumulative rainfall thresholds

In [ ]:
Crain = pd.DataFrame(index=rainls.index)

cmltv = [0, 175, 42, 49, 84]
for C in cmltv:
    Crain[f"c{str(C)}"] = rainls["rain1"].shift(-C)

# addding labels to samples (rows)
Crain["den"] = rainls.den
#removing NaNs as result of cumulative and lag functions
Crain = Crain.dropna(axis=0, how="any")

In [ ]:
from scipy.stats import scoreatpercentile
# define percentiles
minpct = 1
maxpct = 99
steps = 100
rainpct = np.linspace(minpct, maxpct, steps)
# define dataframes for storing thresholds and score-at-percentiles
thresholds = pd.DataFrame(index=rainpct, columns=Crain.columns[:-1])
tprs = pd.DataFrame(index=rainpct, columns=Crain.columns[:-1])
fprs = pd.DataFrame(index=rainpct, columns=Crain.columns[:-1])
aucs = pd.DataFrame(index=rainpct, columns=Crain.columns[:-1])
# SCATTER PLOT
# define figure parameters
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

# Plot results of ML algorithms
marker = "o"
markersize = 10
linewidth = 0
mfc = "None"
mew = 2
# alpha = 0.5
alpha = 0.7
# convert results to percent (0-100)
for cN in _FPRs.columns:
    # plot results of current algorithm
    ax.plot(_FPRs[cN].values, _TPRs[cN].values, marker=marker, mew=mew, mfc=mfc, markersize=markersize, linewidth=linewidth, alpha=alpha, label=cN)
    
alpha = 1
lw = 1

# Loop through cumulative rainfalls
for c in Crain.columns[:-1]:
    # compute score-at-percentiles for the current cumulative rainfall
    thresholds[c] = scoreatpercentile(Crain[c].values, thresholds.index)
    
    # loop through thresholds for the current cumulative rainfall
    for t in thresholds.index:
        # evaluate rainfall exceedence of the current threshold
        predict = np.where(Crain[c].values >= thresholds.loc[t, c], 1, 0)
        # evaluate predictive performance
        tn, fp, fn, tp = confusion_matrix(Crain["den"].values, predict).ravel()
        tpr = tp / (tp+fn)
        fpr = fp / (fp+tn)
        #store results to dataframes
        tprs.loc[t, c] = tpr
        fprs.loc[t, c] = fpr
        aucs.loc[t, c] = auc([0, fpr, 1], [0, tpr, 1])
        
    ax.plot(100 * fprs.loc[:, c], 100 * tprs.loc[:, c], lw=lw, label=f"{c} {str(int(100 * aucs.loc[:, c].max()))}", alpha=alpha)
    
#add random guess line
ax.plot([0, 100], [0, 100], "k:", label="random guess")

# format plot
ax.legend(loc="lower right", ncol=2)
ax.set_ylabel("TPR, %", fontsize="large")
ax.set_xlabel("FPR, %", fontsize="large")
ax.set_xlim(-5, 105)
ax.set_ylim(-5, 105)
plt.savefig("ROCplotML+cmltvmeanmaxminrel.png")

In [ ]:
# TABULAR RESULTS
aucsum = pd.DataFrame()
aucsum["mean"] = aucs.mean()
aucsum["std"] = aucs.std()
aucsum["min"] = aucs.min()
aucsum["maxn"] = aucs.max()

tprsum = pd.DataFrame()
tprsum["mean"] = tprs.mean()
tprsum["std"] = tprs.std()
tprsum["min"] = tprs.min()
tprsum["maxn"] = tprs.max()

fprsum = pd.DataFrame()
fprsum["mean"] = fprs.mean()
fprsum["std"] = fprs.std()
fprsum["min"] = fprs.min()
fprsum["maxn"] = fprs.max()

print("False Positives:")
print(fprsum)
print()
print("True Positives:")
print(tprsum)
print()
print("Aucsum:")
print(aucsum)

# revert results to percent (0.0 - 1.0)
fprs = fprs/100
tprs = tprs/100
aucs = aucs/100

In [ ]:
# define percentiles
minpct = 1
maxpct = 99
steps = 30
rainpct = np.linspace(minpct, maxpct, steps)
# define dataframes for storing thresholds and score-at percentiles
tprs = pd.DataFrame(index=rainpct,columns=Crain.columns[:-1])
fprs = pd.DataFrame(index=rainpct,columns=Crain.columns[:-1])
aucs = pd.DataFrame(index=rainpct,columns=Crain.columns[:-1])
# SCATTER PLOT
# define figure parameters
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

# Plot results of ML algorithms
marker = "o"
markersize = 10
linewidth = 0
mfc = "None"
mew = 2
alpha = 0.7

for cN in _FPRs.columns:
    ax.plot(_FPRs[cN].values, _TPRs[cN].values, marker=marker, mew=mew, mfc=mfc, markersize=markersize, linewidth=linewidth, alpha=alpha, label=cN)

alpha = 1
lw = 1

# add random guess line
ax.plot([0, 100], [0, 100], "k:", label="random guess")

# format plot
ax.legend(loc="lower right", ncol=2)
ax.set_ylabel("TPR, %", fontsize="large")
ax.set_xlabel("FPR, %", fontsize="large")
ax.set_xlim(-5, 105)
ax.set_ylim(-5, 105)
plt.savefig("ROCplotML.png")